<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/scoring_lpips_lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring
This notebook takes the output test restults from the models and calculates their frechet inception distance

In [1]:
!git clone https://github.com/richzhang/PerceptualSimilarity
!pip install -r PerceptualSimilarity/requirements.txt

Cloning into 'PerceptualSimilarity'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 778 (delta 24), reused 28 (delta 12), pack-reused 731
Receiving objects: 100% (778/778), 8.76 MiB | 12.39 MiB/s, done.
Resolving deltas: 100% (455/455), done.


In [2]:
import shutil
import os
import glob
import pandas as pd
import cv2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
test_output_lama = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/lama.zip'
test_dataset = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test.zip'

temp_dir_1 = '/content/temp_folder_1/'
staging_dir = '/content/staging/'
temp_dir_2 = '/content/temp_folder_2/'

In [4]:
def unzip_images(input_zip):
    print(input_zip)

    print('Removing old folder')
    if os.path.isdir(temp_dir_1):
        shutil.rmtree(temp_dir_1)

    print('Unpacking zip file')
    shutil.copy(input_zip, '/content/')
    shutil.unpack_archive(f'/content/{os.path.basename(input_zip)}', staging_dir)

    file_list = glob.glob(temp_dir_1 + '*')
    print(f'Number of images: {len(file_list)}')

    # unpacks the test imagse if they are not already there
    if os.path.isdir(temp_dir_2) == False:
        shutil.copy(test_dataset, '/content/')
        shutil.unpack_archive(f'/content/{os.path.basename(test_dataset)}', temp_dir_2)
    

def rename_files(need_leading_zero=True):
    # rename the files
    temp_1_list = glob.glob(f'{temp_dir_1}*')
    temp_2_list = glob.glob(f'{temp_dir_2}*')

    temp_1_list.sort()
    temp_2_list.sort()

    print('Renaming files')

    if need_leading_zero:
        # have to copy and rename the files
        for i in range(len(temp_1_list)):
            new_file_name = os.path.basename(temp_1_list[i])
            new_file_name = new_file_name.split('_')[1]
            n_digits = len(new_file_name) - 4
            new_file_name = '0' * (5 - n_digits) + new_file_name

            # rename the file
            os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')    

    else:
        # have to copy and rename the files
        for i in range(len(temp_1_list)):
            new_file_name = str(i) + '.png'
            n_digits = len(new_file_name) - 4
            new_file_name = '0' * (5 - n_digits) + new_file_name

            # rename the file
            os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')

    # have to copy and rename test files
    for i in range(len(temp_2_list)):
        new_file_name = str(i) + '.jpg'
        n_digits = len(new_file_name) - 4
        new_file_name = '0' * (5 - n_digits) + new_file_name

        # rename the file
        os.rename(temp_2_list[i], f'{temp_dir_2}{new_file_name}')


def get_score(gpu=True):

    print('Calculating score')
    if gpu:
        os.system('python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt --use_gpu')
    else:
        os.system('python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt')

    df = pd.read_csv('/content/PerceptualSimilarity/imgs/example_dists.txt', sep=' ', header=None)
    print(df.iloc[:, 1].mean())

In [5]:
unzip_images(test_output_lama)

/content/gdrive/MyDrive/repos/GANime/data_out/test_output/lama.zip
Removing old folder
Unpacking zip file
Number of images: 0


In [9]:
file_list = glob.glob(staging_dir + '*')
file_list.sort()

# have to copy and rename the files
for i in range(len(file_list)):
    new_file_name = str(i) + '.jpg'
    n_digits = len(new_file_name) - 4
    new_file_name = '0' * (5 - n_digits) + new_file_name

    img = cv2.imread(file_list[i])
    cv2.imwrite(temp_dir_1 + new_file_name, img)

    if i % 1500 == 0:
        print(i)

0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000
19500
21000
22500
24000
25500
27000
28500
30000
31500
33000
34500
36000
37500
39000
40500
42000
43500
45000
46500
48000
49500
51000
52500
54000
55500
57000
58500
60000


In [7]:
# rename the files
temp_2_list = glob.glob(f'{temp_dir_2}*')
temp_2_list.sort()

print('Renaming test files ')

# have to copy and rename test files
for i in range(len(temp_2_list)):
    new_file_name = str(i) + '.jpg'
    n_digits = len(new_file_name) - 4
    new_file_name = '0' * (5 - n_digits) + new_file_name

    # rename the file
    os.rename(temp_2_list[i], f'{temp_dir_2}{new_file_name}')

Renaming test files 


In [ ]:
# get_score()
!python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt
df = pd.read_csv('/content/PerceptualSimilarity/imgs/example_dists.txt', sep=' ', header=None)
print(df.iloc[:, 1].mean())

Streaming output truncated to the last 5000 lines.
41413.jpg: 0.085
55038.jpg: 0.111
55009.jpg: 0.097
59991.jpg: 0.057
47417.jpg: 0.094
42924.jpg: 0.104
59393.jpg: 0.148
59594.jpg: 0.119
08219.jpg: 0.108
48515.jpg: 0.082
19952.jpg: 0.088
31176.jpg: 0.107
40303.jpg: 0.070
51329.jpg: 0.140
46994.jpg: 0.093
11898.jpg: 0.124
49054.jpg: 0.129
57763.jpg: 0.067
29261.jpg: 0.074
60493.jpg: 0.048
25755.jpg: 0.084
08693.jpg: 0.121
45929.jpg: 0.068
39129.jpg: 0.106
01029.jpg: 0.140
39550.jpg: 0.115
51148.jpg: 0.128
34017.jpg: 0.108
15059.jpg: 0.118
35038.jpg: 0.057
20049.jpg: 0.091
60925.jpg: 0.092
41422.jpg: 0.137
19533.jpg: 0.145
14604.jpg: 0.109
15269.jpg: 0.111
04265.jpg: 0.128
18596.jpg: 0.101
26791.jpg: 0.145
54100.jpg: 0.056
39348.jpg: 0.103
35288.jpg: 0.097
40252.jpg: 0.082
60253.jpg: 0.095
46783.jpg: 0.126
10467.jpg: 0.107
15876.jpg: 0.108
48717.jpg: 0.119
60356.jpg: 0.128
12298.jpg: 0.099
40025.jpg: 0.122
51644.jpg: 0.118
32617.jpg: 0.085
54555.jpg: 0.141
56045.jpg: 0.099
24930.jpg: 0.1